In [1]:
def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        assert data, f"Файл {file_path} пустой или не удалось cчитать данные."
        return data
        
text = read_data('C:/Tokenizer/DataSets/en.txt')

In [2]:
import re

def clean_text(text):
    text = re.sub(r"[^a-z\s]", "", text.lower()) 
    text = ' '.join(text.split())
    return text
    
corpus = clean_text(text)

In [3]:
if corpus is not None:
    print("Первые 100 символов из анг:")
    print(corpus[:100])

Первые 100 символов из анг:
stopping russia starts in syria washington dc the solution to the crisis in ukraine lies in part in 


In [4]:
from collections import defaultdict

vocab = defaultdict(int)
for word in corpus.split():
    for char in word:
        vocab[char] += 1
    vocab['_ed'] += 1

In [5]:
print(len(vocab))
vocab

27


defaultdict(int,
            {'s': 4793736,
             't': 6573886,
             'o': 5373890,
             'p': 1593574,
             'i': 5701556,
             'n': 5450891,
             'g': 1456995,
             '_ed': 13746943,
             'r': 4620693,
             'u': 2049723,
             'a': 5825433,
             'y': 1118885,
             'w': 1061333,
             'h': 2969104,
             'd': 2526262,
             'c': 2586146,
             'e': 8707030,
             'l': 3086439,
             'k': 407799,
             'm': 1862158,
             'f': 1482604,
             'b': 1083111,
             'v': 804839,
             'j': 101585,
             'x': 146008,
             'q': 71929,
             'z': 92544})

In [6]:
words = defaultdict(int)

for word in corpus.split():
    symbols = list(word) + ['_ed']
    words[' '.join(symbols)] += 1

In [7]:
for i, (key, value) in enumerate(words.items()):
    if i < 5:
        print(f"'{key}': {value}")
    else:
        break

's t o p p i n g _ed': 215
'r u s s i a _ed': 8649
's t a r t s _ed': 331
'i n _ed': 305742
's y r i a _ed': 2903


In [8]:
def get_stats(words):
    pairs = defaultdict(int)
    for word, freq in words.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            if symbols[i+1] != '_ed':
                pair = (symbols[i], symbols[i+1])
                pairs[pair] += freq
    return pairs

In [9]:
pairs = get_stats(words)
print(sorted(((v,k) for k,v in pairs.items()), reverse=True))

[(1744564, ('t', 'h')), (1480921, ('i', 'n')), (1321614, ('h', 'e')), (1154993, ('a', 'n')), (1108179, ('r', 'e')), (1055052, ('e', 'r')), (1037991, ('o', 'n')), (870594, ('a', 't')), (860007, ('e', 's')), (850211, ('e', 'n')), (825842, ('t', 'i')), (741413, ('o', 'r')), (697103, ('a', 'l')), (696409, ('n', 'd')), (693173, ('i', 't')), (682068, ('n', 't')), (673173, ('s', 't')), (669088, ('t', 'e')), (636678, ('i', 's')), (615880, ('a', 'r')), (603142, ('n', 'g')), (598975, ('t', 'o')), (578063, ('e', 'd')), (566500, ('h', 'a')), (548934, ('c', 'o')), (532193, ('v', 'e')), (506433, ('d', 'e')), (490804, ('i', 'o')), (480608, ('a', 's')), (465744, ('l', 'e')), (461600, ('i', 'c')), (461471, ('o', 'f')), (460539, ('s', 'e')), (457865, ('r', 'o')), (455772, ('l', 'i')), (455569, ('m', 'e')), (454984, ('r', 'i')), (450786, ('e', 'a')), (443401, ('o', 'u')), (425610, ('r', 'a')), (390796, ('n', 'e')), (369756, ('e', 'c')), (360771, ('c', 'e')), (354989, ('o', 'm')), (344939, ('n', 's')), (3

In [10]:
top_pair = max(pairs, key=pairs.get)
top_pair

('t', 'h')

In [11]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = ' '.join(pair)
    pattern = re.compile(r'(?<!\S)' + re.escape(bigram) + r'(?!\S)')
    for word_in in v_in:
        word_out = pattern.sub(''.join(pair), word_in)
        v_out[word_out] = v_in[word_in]
    return v_out

In [12]:
vocab_size = 1000
num_merges = vocab_size - len(vocab)
for i in range(num_merges):
    pairs = get_stats(words)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab[''.join(best)] = pairs[best]
    words = merge_vocab(best, words)
    if i % 10 == 0:
        print(f'Итерация {i}: слияние {best}, размер словаря: {len(vocab)}')
print(f'Обучение завершено. Размер словаря: {len(vocab)}')

Итерация 0: слияние ('t', 'h'), размер словаря: 28
Итерация 10: слияние ('s', 't'), размер словаря: 38
Итерация 20: слияние ('i', 'c'), размер словаря: 48
Итерация 30: слияние ('i', 'l'), размер словаря: 58
Итерация 40: слияние ('th', 'at'), размер словаря: 68
Итерация 50: слияние ('u', 't'), размер словаря: 78
Итерация 60: слияние ('p', 'e'), размер словаря: 88
Итерация 70: слияние ('re', 's'), размер словаря: 98
Итерация 80: слияние ('k', 'e'), размер словаря: 108
Итерация 90: слияние ('es', 's'), размер словаря: 118
Итерация 100: слияние ('i', 'al'), размер словаря: 128
Итерация 110: слияние ('p', 'ol'), размер словаря: 138
Итерация 120: слияние ('l', 'i'), размер словаря: 148
Итерация 130: слияние ('mo', 're'), размер словаря: 158
Итерация 140: слияние ('ic', 'al'), размер словаря: 168
Итерация 150: слияние ('the', 'ir'), размер словаря: 178
Итерация 160: слияние ('i', 'a'), размер словаря: 188
Итерация 170: слияние ('ch', 'in'), размер словаря: 198
Итерация 180: слияние ('wor', 'l

In [13]:
vocab

defaultdict(int,
            {'s': 4793736,
             't': 6573886,
             'o': 5373890,
             'p': 1593574,
             'i': 5701556,
             'n': 5450891,
             'g': 1456995,
             '_ed': 13746943,
             'r': 4620693,
             'u': 2049723,
             'a': 5825433,
             'y': 1118885,
             'w': 1061333,
             'h': 2969104,
             'd': 2526262,
             'c': 2586146,
             'e': 8707030,
             'l': 3086439,
             'k': 407799,
             'm': 1862158,
             'f': 1482604,
             'b': 1083111,
             'v': 804839,
             'j': 101585,
             'x': 146008,
             'q': 71929,
             'z': 92544,
             'th': 1744564,
             'in': 1480921,
             'an': 1154993,
             'the': 1139668,
             're': 1108179,
             'on': 1037991,
             'at': 848723,
             'er': 846127,
             'en': 772314,
         

In [14]:
print(len(vocab))

1000


In [15]:
def build_token_mappings(vocab):
    tokens = list(vocab.keys())
    token_to_id = {token: idx for idx, token in enumerate(tokens)}
    id_to_token = {idx: token for idx, token in enumerate(tokens)}
    return token_to_id, id_to_token

token_to_id, id_to_token = build_token_mappings(vocab)

In [16]:
def add_special_tokens(token_to_id, id_to_token, special_tokens):
    for token in special_tokens:
        if token not in token_to_id:
            idx = len(token_to_id)
            token_to_id[token] = idx
            id_to_token[idx] = token

special_tokens = ['<start>', '<end>', '<unk>', '<pad>']
add_special_tokens(token_to_id, id_to_token, special_tokens)

print(f"\nДобавлены специальные токены. Размер словаря token_to_id: {len(token_to_id)}")


Добавлены специальные токены. Размер словаря token_to_id: 1004


In [17]:
import json
import os

output_dir = "C:/Tokenizer/Vocab/EnVocab" 

os.makedirs(output_dir, exist_ok=True)

with open(os.path.join(output_dir, 'token_to_id.json'), 'w', encoding='utf-8') as out_file:
    json.dump(token_to_id, out_file, indent=6, ensure_ascii=False)

with open(os.path.join(output_dir, 'id_to_token.json'), 'w', encoding='utf-8') as out_file:
    json.dump(id_to_token, out_file, indent=6, ensure_ascii=False)

print(f"Словари token_to_id и id_to_token сохранены в {output_dir}")


Словари token_to_id и id_to_token сохранены в C:/Tokenizer/Vocab/EnVocab


In [18]:
def tokenize_word(word, vocab):
    symbols = list(word) + ['_ed']  
    i = 0
    tokens = []
    while i < len(symbols):
        j = len(symbols)
        found = False
        while j > i:
            substring = ''.join(symbols[i:j])
            if substring in vocab:
                tokens.append(substring)
                i = j
                found = True
                break
            else:
                j -= 1
        if not found:
            tokens.append('<unk>')
            i += 1
    return tokens

In [19]:
def encode_text(text, token_to_id, vocab):
    token_ids = []
    for word in text.split():
        word_tokens = tokenize_word(word, vocab)
        for token in word_tokens:
            token_id = token_to_id.get(token)
            if token_id is not None:
                token_ids.append(token_id)
            else:
                token_ids.append(token_to_id.get('<unk>'))
    return token_ids

In [20]:
def decode_ids(token_ids, id_to_token):
    tokens = [id_to_token.get(token_id, '<unk>') for token_id in token_ids]
    words = []
    word = ''
    for token in tokens:
        if token == '_ed':
            words.append(word)
            word = ''
        elif token not in ['<start>', '<end>', '<pad>']:
            word += token
    if word:
        words.append(word)
    return ' '.join(words)

In [21]:
test_text = "I study in technical university, because i want to be a programmer"
test_text = clean_text(test_text)

In [22]:
encoded_ids = encode_text(test_text, token_to_id, vocab)
print("\nКодированные идентификаторы:")
print(encoded_ids)


Кодированные идентификаторы:
[4, 7, 37, 280, 11, 7, 28, 7, 643, 167, 7, 71, 126, 8, 246, 340, 7, 481, 7, 4, 7, 12, 178, 7, 43, 7, 62, 7, 10, 7, 956, 190, 8, 7]


In [23]:
decoded_text = decode_ids(encoded_ids, id_to_token)
print("\nДекодированный текст:")
print(decoded_text)


Декодированный текст:
i study in technical university because i want to be a programmer
